In [31]:
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, ZeroPadding2D
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical

import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import random
import cv2 as cv
import numpy as np

In [32]:
DATADIR = 'dataset'
dirs = []
training_data = []
width, height = 13, 19

# Looping direktori data training untuk diambil nama karakternya
for char_name in sorted(os.listdir(DATADIR)):
    dirs.append(char_name)

# Looping semua image data training untuk diubah menjadi array
for char_name in dirs:
    path = os.path.join(DATADIR, char_name)
    class_number = dirs.index(char_name)

    for img in tqdm(os.listdir(path)):
        try:
            img_array = cv.imread(os.path.join(path, img), cv.IMREAD_GRAYSCALE)
            new_array = cv.resize(img_array, (width, height))
            training_data.append([new_array, class_number])
        except Exception as e:
            pass

  0%|          | 0/27 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 1485.24it/s]


In [33]:
random.shuffle(training_data)

X = []
Y = []

for features, label in training_data:
  X.append(features)
  Y.append(label)

X = np.array(X).reshape(-1, width, height, 1)

# Tulis ke file pickle
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("Y.pickle", "wb")
pickle.dump(Y, pickle_out)
pickle_out.close()

In [34]:
# Load file pickle
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("Y.pickle", "rb")
Y = pickle.load(pickle_in)
Y = to_categorical(Y)
X = X / 255.0
# Input layer
inputs = Input(shape=(width, height, 1))
conv_layer = ZeroPadding2D(padding=(2, 2))(inputs)
conv_layer = Conv2D(16, (5, 5), strides=(1, 1), activation='relu')(conv_layer)
conv_layer = MaxPooling2D((2, 2))(conv_layer)
conv_layer = Conv2D(32, (3, 3), strides=(1, 1), activation='relu')(conv_layer)
conv_layer = Conv2D(32, (3, 3), strides=(1, 1), activation='relu')(conv_layer)
conv_layer = MaxPooling2D((2, 2))(conv_layer)
# conv_layer = Conv2D(64, (3, 3), strides=(1, 1), activation='relu')(conv_layer)

flaten = Flatten()(conv_layer)
fc_layer = Dense(256, activation='relu')(flaten)
fc_layer = Dense(64, activation='relu')(fc_layer)

# Output layer
outputs = Dense(len(dirs), activation='softmax')(fc_layer)
adam = Adam(learning_rate=0.0001)
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X, Y, epochs=len(dirs), verbose=1)
model.save('alphabet.model')

Epoch 1/150
66/66 [==============================] - 3s 11ms/step - loss: 4.9825 - accuracy: 0.0346
Epoch 2/150
66/66 [==============================] - 1s 11ms/step - loss: 4.5898 - accuracy: 0.1221
Epoch 3/150
66/66 [==============================] - 1s 13ms/step - loss: 4.0295 - accuracy: 0.1221
Epoch 4/150
66/66 [==============================] - 1s 13ms/step - loss: 3.8307 - accuracy: 0.1221
Epoch 5/150
66/66 [==============================] - 1s 10ms/step - loss: 3.7487 - accuracy: 0.1221
Epoch 6/150
66/66 [==============================] - 1s 11ms/step - loss: 3.7119 - accuracy: 0.1221
Epoch 7/150
66/66 [==============================] - 1s 11ms/step - loss: 3.6935 - accuracy: 0.1221
Epoch 8/150
66/66 [==============================] - 1s 11ms/step - loss: 3.6792 - accuracy: 0.1221
Epoch 9/150
66/66 [==============================] - 1s 11ms/step - loss: 3.6667 - accuracy: 0.1221
Epoch 10/150
66/66 [==============================] - 1s 11ms/step - loss: 3.6517 - accuracy: 0.1221

INFO:tensorflow:Assets written to: alphabet.model\assets
